<a href="https://colab.research.google.com/github/MthabisiPatrice/Machine-Learning-/blob/main/clothing_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
DL Project: Fashion MNIST Clothing Classification (Template)
- Trains 3 model variants (Baseline CNN, Deeper CNN, CNN+Dropout+BatchNorm)
- Produces comparison table + plots
"""

import os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [4]:
# ----------------------
# Reproducibility
# ----------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [5]:
# ----------------------
# Config
# ----------------------
CFG = {
    "batch_size": 128,
    "epochs": 8,
    "patience": 2,
    "models": ["baseline", "deeper", "dropout_bn"],
}

OUTDIR = "outputs_fashion_" + datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(OUTDIR, exist_ok=True)


In [6]:
# ----------------------
# Data Loading
# ----------------------
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize + reshape
x_train = (x_train / 255.0).astype(np.float32).reshape(-1, 28, 28, 1)
x_test  = (x_test / 255.0).astype(np.float32).reshape(-1, 28, 28, 1)

# Split train/val
val_frac = 0.1
n_val = int(len(x_train) * val_frac)
x_val, y_val = x_train[:n_val], y_train[:n_val]
x_train, y_train = x_train[n_val:], y_train[n_val:]

num_classes = 10


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
# ----------------------
# Model Definitions
# ----------------------

def build_baseline():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


def build_deeper():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


def build_dropout_bn():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


In [8]:
# ----------------------
# Training Utility
# ----------------------

def train_and_eval(model_fn, name):
    cbs = [
        callbacks.EarlyStopping(patience=CFG["patience"], restore_best_weights=True, monitor="val_accuracy"),
        callbacks.ModelCheckpoint(os.path.join(OUTDIR, f"best_{name}.keras"), save_best_only=True, monitor="val_accuracy"),
    ]
    model = model_fn()
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=CFG["epochs"],
        batch_size=CFG["batch_size"],
        callbacks=cbs,
        verbose=2,
    )

    # Evaluate
    y_pred = np.argmax(model.predict(x_test), axis=1)
    acc = accuracy_score(y_test, y_pred)

    report = classification_report(y_test, y_pred, digits=4)
    with open(os.path.join(OUTDIR, f"report_{name}.txt"), "w") as f:
        f.write(report)

    # Plot curves
    plt.figure()
    plt.plot(history.history["accuracy"], label="train_acc")
    plt.plot(history.history["val_accuracy"], label="val_acc")
    plt.title(f"Accuracy – {name}")
    plt.legend()
    plt.savefig(os.path.join(OUTDIR, f"acc_{name}.png"))
    plt.close()

    return {"model": name, "test_accuracy": round(acc,4)}


In [9]:
# ----------------------
# Main
# ----------------------

def main():
    results = []
    for m in CFG["models"]:
        if m == "baseline":
            res = train_and_eval(build_baseline, m)
        elif m == "deeper":
            res = train_and_eval(build_deeper, m)
        elif m == "dropout_bn":
            res = train_and_eval(build_dropout_bn, m)
        results.append(res)

    df = pd.DataFrame(results).sort_values("test_accuracy", ascending=False)
    df.to_csv(os.path.join(OUTDIR, "summary.csv"), index=False)
    print("\n=== Summary ===\n", df)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/8
422/422 - 25s - 59ms/step - accuracy: 0.8292 - loss: 0.4955 - val_accuracy: 0.8727 - val_loss: 0.3640
Epoch 2/8
422/422 - 21s - 49ms/step - accuracy: 0.8860 - loss: 0.3246 - val_accuracy: 0.8893 - val_loss: 0.3145
Epoch 3/8
422/422 - 44s - 104ms/step - accuracy: 0.9003 - loss: 0.2811 - val_accuracy: 0.8980 - val_loss: 0.2911
Epoch 4/8
422/422 - 39s - 94ms/step - accuracy: 0.9104 - loss: 0.2516 - val_accuracy: 0.9033 - val_loss: 0.2775
Epoch 5/8
422/422 - 40s - 94ms/step - accuracy: 0.9181 - loss: 0.2291 - val_accuracy: 0.9078 - val_loss: 0.2650
Epoch 6/8
422/422 - 42s - 100ms/step - accuracy: 0.9250 - loss: 0.2104 - val_accuracy: 0.9122 - val_loss: 0.2579
Epoch 7/8
422/422 - 20s - 49ms/step - accuracy: 0.9314 - loss: 0.1940 - val_accuracy: 0.9152 - val_loss: 0.2524
Epoch 8/8
422/422 - 42s - 100ms/step - accuracy: 0.9370 - loss: 0.1790 - val_accuracy: 0.9160 - val_loss: 0.2521
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/8
422/422 - 120s - 285ms/step - accuracy: 0.8425 - loss: 0.4480 - val_accuracy: 0.8900 - val_loss: 0.3148
Epoch 2/8
422/422 - 138s - 328ms/step - accuracy: 0.8998 - loss: 0.2784 - val_accuracy: 0.9068 - val_loss: 0.2608
Epoch 3/8
422/422 - 113s - 269ms/step - accuracy: 0.9176 - loss: 0.2271 - val_accuracy: 0.9143 - val_loss: 0.2379
Epoch 4/8
422/422 - 142s - 336ms/step - accuracy: 0.9321 - loss: 0.1895 - val_accuracy: 0.9185 - val_loss: 0.2288
Epoch 5/8
422/422 - 143s - 339ms/step - accuracy: 0.9436 - loss: 0.1574 - val_accuracy: 0.9215 - val_loss: 0.2304
Epoch 6/8
422/422 - 141s - 334ms/step - accuracy: 0.9549 - loss: 0.1297 - val_accuracy: 0.9188 - val_loss: 0.2426
Epoch 7/8
422/422 - 113s - 267ms/step - accuracy: 0.9634 - loss: 0.1062 - val_accuracy: 0.9172 - val_loss: 0.2637
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/8
422/422 - 173s - 410ms/step - accuracy: 0.7789 - loss: 0.6427 - val_accuracy: 0.7768 - val_loss: 1.0500
Epoch 2/8
422/422 - 204s - 484ms/step - accuracy: 0.8452 - loss: 0.4295 - val_accuracy: 0.8922 - val_loss: 0.3004
Epoch 3/8
422/422 - 201s - 476ms/step - accuracy: 0.8685 - loss: 0.3628 - val_accuracy: 0.8957 - val_loss: 0.2821
Epoch 4/8
422/422 - 199s - 472ms/step - accuracy: 0.8836 - loss: 0.3224 - val_accuracy: 0.9037 - val_loss: 0.2585
Epoch 5/8
422/422 - 165s - 391ms/step - accuracy: 0.8943 - loss: 0.2888 - val_accuracy: 0.9100 - val_loss: 0.2471
Epoch 6/8
422/422 - 205s - 486ms/step - accuracy: 0.8989 - loss: 0.2741 - val_accuracy: 0.8982 - val_loss: 0.2863
Epoch 7/8
422/422 - 198s - 470ms/step - accuracy: 0.9049 - loss: 0.2574 - val_accuracy: 0.9173 - val_loss: 0.2380
Epoch 8/8
422/422 - 205s - 485ms/step - accuracy: 0.9125 - loss: 0.2379 - val_accuracy: 0.9162 - val_loss: 0.2356
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step

=== Summary ===
         model  test_accuracy